**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [3]:
!pip install opencv-contrib-python
!sudo pip install scikit-video

In [0]:
import tensorflow.keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from tensorflow.keras.models import Sequential,model_from_json
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization,Flatten

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

Given the current state, the funcion act returns the action the agent makes in the environment (After that, he will observe the next state and reward  from the environment). 

In the traininig phase, the agent choose either to explore the environment with a probability $\epsilon$ by taking a random action, or to exploit what he has learned so far with a pobability of $1-\epsilon$ (using greedy strategy for example). This is what is known by exploitation-exploration dilemma. Using this $\epsilon$ allows the agent to discover more and new things about the environement  by giving a chance to all the actions to improve the way they are perceived so far and avoid getting stuck in taking some specific actions (these that were taken the most and that give the best rewards). In other words, $\epsilon$ quantifies somehow how much to exploit and how much to explore the environmenet.

During the test phase, the agent samples the best action based on what he has learned given the current state. 


***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=50 # set small when debugging
epochs_test=20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

position is an array of size $(grid\_size \times grid\_size)$ that gives information about the position of the agent, where he cannot go (forbidden states) and the allowed states. The value is 1 where the rat is, $-1$ in the boundaries where we can't go ( $2\times 2$ padding in the border of the grid ), and 0 everywhere else (allowed states). 

board is an array of size $(grid\_size \times grid\_size)$ that defines the spatial distribution of the rewards on the grid. $board[x,y]$ represents the reward of getting in position (x,y). $board[x, y]=0$ if there's neither bonus nor malus in cell (x,y). It equals 0.5 if there's a cheese in the cell, -1 otherwise ( poisonous cell ). We set the reward to 0 if it is collected, i.e the state has already been visited ($\to$ which means that board evolves over time). 

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0        
    for e in range(epochs):
        state = env.reset()
        game_over = False  # True when the game will end
        win = 0 
        lose = 0
        while not game_over :
            # The agent takes an action
            action = agent.act(state,train=False)
            state, reward, game_over = env.act(action)    # observe the next state and reward
            # positive reward : win ----- negative reward : loss. 
            if reward > 0 :
                win += reward
            else :
                lose -= reward
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [11]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 11.5/13.0. Average score (-1.5)
Win/lose count 6.5/15.0. Average score (-5.0)
Win/lose count 10.5/18.0. Average score (-5.833333333333333)
Win/lose count 7.5/15.0. Average score (-6.25)
Win/lose count 8.0/11.0. Average score (-5.6)
Win/lose count 10.0/12.0. Average score (-5.0)
Win/lose count 12.0/22.0. Average score (-5.714285714285714)
Win/lose count 11.5/16.0. Average score (-5.5625)
Win/lose count 9.0/15.0. Average score (-5.611111111111111)
Win/lose count 11.0/9.0. Average score (-4.85)
Win/lose count 6.5/10.0. Average score (-4.7272727272727275)
Win/lose count 14.5/14.0. Average score (-4.291666666666667)
Win/lose count 9.5/12.0. Average score (-4.153846153846154)
Win/lose count 10.5/17.0. Average score (-4.321428571428571)
Win/lose count 11.5/13.0. Average score (-4.133333333333334)
Win/lose count 11.5/18.0. Average score (-4.28125)
Win/lose count 11.5/11.0. Average score (-4.0)
Win/lose count 9.0/15.0. Average score (-4.111111111111111)
Win/lose count 9.0/17.0. A

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




1. We have : 

 $$
\begin{align*} 
Q^\pi(s,a)&=E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]\\
          &=E_{p^{\pi}}[r(s_0,a_0) + \gamma\sum_{t=1}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]\\
          &=r(s,a) + \sum_{s'} p(s',a'|s,a) E_{p^{\pi}}[\sum_{t=1}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_1 = s', a_1 = a', s_{0}=s,a_{0}=a]\\
          &=r(s,a) + \gamma \sum_{s'} p(s',a'|s,a) E_{p^{\pi}}[\sum_{t=1}^{\infty}\gamma^{t-1}r(s_{t},a_{t})|s_1 = s', a_1 = a']  \text{ $\to$ (Markov property) } \\
          &=r(s,a) + \gamma \sum_{s'} p(s',a'|s,a) Q^\pi(s',a') \\
          &=r(s,a) + \gamma E_{(s',a')\sim p(.|s,a)}( Q^\pi(s',a')) \\
          &=E_{(s',a')\sim p(.|s,a)}\left(r(s,a) + \gamma Q^{\pi}(s',a') \right)\\
\end{align*}
$$

2. Let $\pi^*$ be an optimal policy. Assume that $\pi^*$ is deterministic (indeed if optimal policy exists, there is also a deterministic optimal policy).  Using the fact that $\pi^*$ is optimal ($\pi'$ such that $\pi^*=(a, \pi^{'})$ is optimal) , We have :

 $$
\begin{align*} 
Q^*(s,a)&= Q^{\pi^*}(s,a) \\
        &= \max_{\pi}E_{(s',a')\sim p(.|s,a)}\left(r(s,a) + \gamma Q^{\pi}(s',a') \right)\\
        &= \max_{\pi} E_{s'\sim p(.|s,a), a' \sim \pi(.|s')}\left(r(s,a) + \gamma Q^{\pi}(s',a') \right)\\
        &= E_{s'\sim p(.|s,a), a' \sim \pi^*(.|s')}\left(r(s,a) + \gamma Q^{\pi^*}(s',a') \right) \text{$\to$ $\pi'$ such that $\pi^*=(a, \pi^{'})$ is optimal}\\
        & = E_{s'\sim p(.|s,a)}\left(r(s,a) + \gamma \max_{a'} Q^{\pi^*}(s',a') \right)\text{ $\to$ $\pi^*$ is a determinitic policy } \\
        & = E_{s'\sim p(.|s,a)}\left(r(s,a) + \gamma \max_{a'} Q^{*}(s',a') \right)
\end{align*}\\       
$$

3. Let define the Bellman operator $T$ (which is contractant) such that: 
 $$
\begin{align*}
TQ(s,a) &=  E_{s'\sim p(.|s,a)}[r(s,a)+\gamma \max_{a'}Q(s',a')] \\
\end{align*}\\       
$$

$Q^*$ verifies the equation $TQ = Q$. 

So one plausible objective is the one that will enforce our optimal Q-function to satisfy the bellman equation. 

Let parametrise Q by a parameter $\theta$. We are looking to minimize the squared error loss $(Q^*(s,a)-Q_\theta(s,a))^2$. However, $Q^*(s,a)$ is unknown, so we replace it at each step t by an estimate $y_t = TQ_{\theta_{t-1}}$ such that : 
$TQ_{\theta_{t-1}}(s,a) =  E_{s'\sim p(.|s,a)}[r(s,a)+\gamma \max_{a'}Q_{\theta_{t-1}}(s',a')]$. 

One plausible choice is to minimize the quantitiy : $E[(r(s,a) + \gamma \max_{a'}Q_{\theta_{t-1}}(s',a') - Q_{\theta_{t}}(s,a))^2]$. As we are working in a continuous setting ( DQN : using gradient descent to solve the problem), we can switch from the discret $\theta_t$ to a continuous $\theta$. 

Hence, we deduce that : 

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

is a plausible objective function for this task. 

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
"""Here, I used a collections.deque as it can deal with max_memory by deleating the eldest element when 
            the max memory is reached and replacing it by the new arriving element. """
from collections import deque


class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = deque(maxlen = self.max_memory) 

    def remember(self, m):
        self.memory.append(m)

    def random_access(self):
        random_idx = np.random.randint(len(self.memory))
        return self.memory[random_idx]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))
            
        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size


    def learned_act(self, s):
        return np.argmax(self.model.predict(s[None]))
    
    
    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            
            # store the sampled state from the replay buffer
            input_states[i] = s_ 
            # Its associated Q-value
            target_q[i] = self.model.predict(s_[None])
            
            if game_over_:
                ######## FILL IN
                target_q[i, a_] = r_
            else:
                ######## FILL IN
                target_q[i, a_] = r_ + self.discount* np.max(self.model.predict(n_s_[None]), axis = 1)

        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        ####### FILL IN
        model = Sequential()
        model.add(Flatten(input_shape=(5, 5, self.n_state, )))
        model.add(Dense(48,activation ='relu')) 
        model.add(Dense(4))
        
        
        model.compile(SGD(lr=lr, decay=1e-4, momentum=0.0), "mse")
        #model.compile(Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False), "mse")
        self.model = model


In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='fc_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/050 | Loss 0.0028 | Win/lose count 3.5/4.0 (-0.5)
Epoch 001/050 | Loss 0.0023 | Win/lose count 1.0/6.0 (-5.0)
Epoch 002/050 | Loss 0.0067 | Win/lose count 3.5/4.0 (-0.5)
Epoch 003/050 | Loss 0.0247 | Win/lose count 5.5/2.0 (3.5)
Epoch 004/050 | Loss 0.0211 | Win/lose count 6.5/6.0 (0.5)
Epoch 005/050 | Loss 0.0234 | Win/lose count 5.5/5.0 (0.5)
Epoch 006/050 | Loss 0.0598 | Win/lose count 4.5/2.0 (2.5)
Epoch 007/050 | Loss 0.0061 | Win/lose count 2.5/3.0 (-0.5)
Epoch 008/050 | Loss 0.0078 | Win/lose count 1.5/2.0 (-0.5)
Epoch 009/050 | Loss 0.0056 | Win/lose count 7.0/3.0 (4.0)
Epoch 010/050 | Loss 0.0582 | Win/lose count 4.0/3.0 (1.0)
Epoch 011/050 | Loss 0.0589 | Win/lose count 6.5/2.0 (4.5)
Epoch 012/050 | Loss 0.0031 | Win/lose count 5.0/6.0 (-1.0)
Epoch 013/050 | Loss 0.0045 | Win/lose count 7.0/1.0 (6.0)
Epoch 014/050 | Loss 0.0052 | Win/lose count 7.0/6.0 (1.0)
Epoch 015/050 | Loss 0.0127 | Win/lose count 4.5/5.0 (-0.5)
Epoch 016/050 | Loss 0.0070 | Win/lose count 11.5

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(64,(2,2),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Conv2D(32,(2,2),activation='relu'))
        model.add(Flatten())
        model.add(Dense(units = 4))
        
        model.compile(SGD(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/050 | Loss 0.0008 | Win/lose count 1.5/4.0 (-2.5)
Epoch 001/050 | Loss 0.0001 | Win/lose count 1.0/1.0 (0.0)
Epoch 002/050 | Loss 0.0010 | Win/lose count 2.0/7.0 (-5.0)
Epoch 003/050 | Loss 0.0014 | Win/lose count 2.0/1.0 (1.0)
Epoch 004/050 | Loss 0.0065 | Win/lose count 6.5/4.0 (2.5)
Epoch 005/050 | Loss 0.0047 | Win/lose count 7.0/3.0 (4.0)
Epoch 006/050 | Loss 0.0102 | Win/lose count 6.5/8.0 (-1.5)
Epoch 007/050 | Loss 0.0029 | Win/lose count 9.0/0 (9.0)
Epoch 008/050 | Loss 0.0017 | Win/lose count 9.5/3.0 (6.5)
Epoch 009/050 | Loss 0.0060 | Win/lose count 5.5/4.0 (1.5)
Epoch 010/050 | Loss 0.0059 | Win/lose count 7.5/1.0 (6.5)
Epoch 011/050 | Loss 0.0016 | Win/lose count 3.5/0 (3.5)
Epoch 012/050 | Loss 0.0017 | Win/lose count 11.0/3.0 (8.0)
Epoch 013/050 | Loss 0.0020 | Win/lose count 8.0/1.0 (7.0)
Epoch 014/050 | Loss 0.0046 | Win/lose count 15.0/3.0 (12.0)
Epoch 015/050 | Loss 0.0019 | Win/lose count 8.5/1.0 (7.5)
Epoch 016/050 | Loss 0.0539 | Win/lose count 15.5/5.0 

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [0]:
env = Environment(grid_size=size, max_time=T,temperature=0.4)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 5.0/1.0. Average score (4.0)
Win/lose count 6.0/0.0. Average score (5.0)
Win/lose count 5.5/0.0. Average score (5.166666666666667)
Win/lose count 7.5/0.0. Average score (5.75)
Win/lose count 8.5/0.0. Average score (6.3)
Win/lose count 5.5/0.0. Average score (6.166666666666667)
Win/lose count 5.0/0.0. Average score (6.0)
Win/lose count 3.0/0.0. Average score (5.625)
Win/lose count 0.5/0.0. Average score (5.055555555555555)
Win/lose count 3.0/0.0. Average score (4.85)
Win/lose count 13.0/0.0. Average score (5.590909090909091)
Win/lose count 13.5/0.0. Average score (6.25)
Win/lose count 5.0/0.0. Average score (6.153846153846154)
Win/lose count 3.5/1.0. Average score (5.892857142857143)
Win/lose count 2.5/0.0. Average score (5.666666666666667)
Win/lose count 3.0/0.0. Average score (5.5)
Win/lose count 2.5/0.0. Average score (5.323529411764706)
Win/lose count 9.5/0.0. Average score (5.555555555555555)
Win/lose count 20.0/0.0. Average score (6.315789473684211)


In [0]:
HTML(display_videos('cnn_test10.mp4'))

In [0]:
HTML(display_videos('fc_test10.mp4'))

The main issue is that the agent lacks exploration of the environment. It tends to quickly get stuck in loops when there is no positive reward(  the rat can't observe any positive reward, it appears especially when the temperature is low).

When increasing the temperature, the score of the agent improves. We notice that he tends to explore more as the temperature gets higher (since positive rewards are almost always visible for him).


The table below summarises some results ( the networks was tranined on temperature = 0.3 )

|Temperature|FC average Test score|CNN Test average score |
|----|-----|----|
|0.1|-0.4|1.25|
|0.3|0.175|3.125|
|0.5|1.34|7.3|
|0.9|2.475|18.025|


The performance using the architecture with CNN is widely better than the one obtained using  fully connected networks. The score of both of them improves with the temperature and so does the gap between the scores of the two architectures. 

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix=''):
    
    # Number of won games
    score = 0
    loss = 0
    
    '''
    We start from a given epsilon and we decrease it exponentially after each epoch to reach 
    approximately 0.1 by the end. 
    "T" will be used to avoid getting a value of 0.1 for epsilon very quickly --> to slow down the 
    decrease of epsilon (favour exploration)
    '''
    start_eps = agent.epsilon
    T = epoch/10   
    
    for e in range(epoch):
        state = env.reset()
        game_over = False

        win = 0
        lose = 0
        eps = (start_eps - 0.1)*np.exp(-e/T) + 0.1
        
        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # reinforce using strategy with the adapted epsilon
            agent.set_epsilon(eps)
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            

        # Save as a mp4
        if (e + 1) % 10 == 0:
            env.draw(prefix+str(e+1))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {:.3f}/{:.3f} ({:.3f}), epsilon {:.3f}"
              .format(e, epoch, loss, win, lose, win-lose, eps))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
    
        
class EnvironmentExploring(object):
  
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size)) #define malus when going to a previously visited position

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))
        
        

    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b



    def act(self, action, train = False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = 0
        is_train = int(train)
        
        reward = self.board[self.x, self.y] - is_train * self.malus_position[self.x, self.y]
        
        # penalisation if we re-visit an already visited state
        penalization = 0.1
        self.malus_position[self.x, self.y] = penalization
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]

        return state, reward, game_over

    
    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        penalization = 0.1
        self.malus_position[self.x, self.y] = penalization

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [0]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.9, memory_size=2000, batch_size = 32, n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/050 | Loss 0.0066 | Win/lose count 8.500/25.400 (-16.900), epsilon 0.900
Epoch 001/050 | Loss 0.0196 | Win/lose count 13.000/26.600 (-13.600), epsilon 0.755
Epoch 002/050 | Loss 0.0094 | Win/lose count 11.000/24.600 (-13.600), epsilon 0.636
Epoch 003/050 | Loss 0.0070 | Win/lose count 14.000/21.800 (-7.800), epsilon 0.539
Epoch 004/050 | Loss 0.0598 | Win/lose count 9.500/22.300 (-12.800), epsilon 0.459
Epoch 005/050 | Loss 0.0079 | Win/lose count 7.500/20.800 (-13.300), epsilon 0.394
Epoch 006/050 | Loss 0.0430 | Win/lose count 7.000/19.000 (-12.000), epsilon 0.341
Epoch 007/050 | Loss 0.0040 | Win/lose count 15.500/21.200 (-5.700), epsilon 0.297
Epoch 008/050 | Loss 0.0090 | Win/lose count 18.000/16.400 (1.600), epsilon 0.262
Epoch 009/050 | Loss 0.0051 | Win/lose count 8.500/19.400 (-10.900), epsilon 0.232
Epoch 010/050 | Loss 0.0077 | Win/lose count 16.000/16.800 (-0.800), epsilon 0.208
Epoch 011/050 | Loss 0.0057 | Win/lose count 21.500/13.000 (8.500), epsilon 0.189
Epoc

In [0]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 21.0/0.0. Average score (21.0)
Win/lose count 12.5/0.0. Average score (16.75)
Win/lose count 15.5/2.0. Average score (15.666666666666666)
Win/lose count 9.5/3.0. Average score (13.375)
Win/lose count 8.0/0.0. Average score (12.3)
Win/lose count 4.0/0.0. Average score (10.916666666666666)
Win/lose count 22.5/0.0. Average score (12.571428571428571)
Win/lose count 25.0/0.0. Average score (14.125)
Win/lose count 23.5/0.0. Average score (15.166666666666666)
Win/lose count 23.0/0.0. Average score (15.95)
Win/lose count 6.0/1.0. Average score (14.954545454545455)
Win/lose count 15.5/0.0. Average score (15.0)
Win/lose count 14.0/0.0. Average score (14.923076923076923)
Win/lose count 14.5/2.0. Average score (14.75)
Win/lose count 4.0/0.0. Average score (14.033333333333333)
Win/lose count 11.5/0.0. Average score (13.875)
Win/lose count 4.0/0.0. Average score (13.294117647058824)
Win/lose count 16.0/0.0. Average score (13.444444444444445)
Win/lose count 5.5/0.0. Average score (13.0

Training the model with the same architecture and using the same temperature (0.3), the performance is widely better now than what we got before.  

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***